In [20]:
import pandas as pd
import re

In [21]:
series = pd.read_excel('SOMC Data SET Clear (1).xlsx', sheet_name='Option Series')
series['END_DATE'] = pd.to_datetime(series['END_DATE'], format='%Y%m%d')
series = series[['INSTRUMENT_ID', 'NAME', 'END_DATE']]
series

,INSTRUMENT_ID,NAME,END_DATE
0,8301,S50K14C1000,2014-05-30
1,8302,S50K14C825,2014-05-30
2,8303,S50K14C850,2014-05-30
3,8304,S50K14C875,2014-05-30
4,8305,S50K14C900,2014-05-30
...,...,...,...
3179,96302,S50N25P675,2025-07-31
3180,96307,S50U25P875,2025-09-30
3181,96304,S50U25C875,2025-09-30
3182,96305,S50N25C875,2025-07-31


In [22]:
settlement = pd.read_excel('EquityIndexOptionsHistTrading-2021-2024.xlsx')
settlement = settlement[['Series', 'Trade Date', 'Settlement']]
settlement

,Series,Trade Date,Settlement
0,S50F21C1000,2021-01-04,4.9
1,S50F21C1025,2021-01-04,2.5
2,S50F21C1050,2021-01-04,1.1
3,S50F21C650,2021-01-04,272.6
4,S50F21C675,2021-01-04,247.6
...,...,...,...
90327,S50M25P875,2024-12-30,19.5
90328,S50M25P900,2024-12-30,30.0
90329,S50M25P925,2024-12-30,45.1
90330,S50M25P950,2024-12-30,62.1


In [23]:
merged_df = pd.merge(series, settlement, left_on='NAME', right_on='Series', how='inner')
merged_df = merged_df[['INSTRUMENT_ID', 'NAME', 'END_DATE', 'Trade Date', 'Settlement']]
merged_df['Trade Date'] = pd.to_datetime(merged_df['Trade Date'], format='%Y-5m-%d')
merged_df

,INSTRUMENT_ID,NAME,END_DATE,Trade Date,Settlement
0,57501,S50H21C775,2021-03-31,2021-01-04,149.5
1,57501,S50H21C775,2021-03-31,2021-01-05,177.8
2,57501,S50H21C775,2021-03-31,2021-01-06,165.3
3,57501,S50H21C775,2021-03-31,2021-01-07,178.9
4,57501,S50H21C775,2021-03-31,2021-01-08,200.0
...,...,...,...,...,...
101577,95969,S50M25C900,2025-06-30,2024-12-30,33.6
101578,95977,S50M25C1000,2025-06-30,2024-12-27,5.2
101579,95977,S50M25C1000,2025-06-30,2024-12-30,5.1
101580,95979,S50M25C975,2025-06-30,2024-12-27,10.4


In [24]:
def create_settlement_price_movement(df, series_col='NAME', trade_date_col='Trade Date', settlement_col='Settlement'):
    """
    Creates a DataFrame showing the settlement price movement of options for each END_DATE.

    Args:
        df (pd.DataFrame): Input DataFrame containing option data.
        series_col (str, optional): Name of the column containing option series. Defaults to 'Series'.
        trade_date_col (str, optional): Name of the column containing trade dates. Defaults to 'Trade Date'.
        settlement_col (str, optional): Name of the column containing settlement prices. Defaults to 'Settlement'.

    Returns:
        pd.DataFrame: A DataFrame showing settlement prices for each option series, grouped by END_DATE.
    """

    # Convert 'END_DATE' and 'Trade Date' to datetime objects
    df['END_DATE'] = pd.to_datetime(df['END_DATE'], errors='coerce')  # Handle potential conversion errors
    df[trade_date_col] = pd.to_datetime(df[trade_date_col], errors='coerce')

    # Group by 'END_DATE' and create a nested structure
    result_df = df.groupby('END_DATE').apply(
        lambda x: x.pivot_table(index=series_col, columns=trade_date_col, values=settlement_col, aggfunc='first')
    ).reset_index()

    return result_df

In [27]:
result_df = create_settlement_price_movement(merged_df)
result_df

/var/folders/rc/w7r40l35143bxwkzfxdygctc0000gn/T/ipykernel_17442/4172367888.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = df.groupby('END_DATE').apply(


Trade Date,END_DATE,NAME,2021-01-04 00:00:00,2021-01-05 00:00:00,2021-01-06 00:00:00,2021-01-07 00:00:00,2021-01-08 00:00:00,2021-01-11 00:00:00,2021-01-12 00:00:00,2021-01-13 00:00:00,...,2024-12-17 00:00:00,2024-12-18 00:00:00,2024-12-19 00:00:00,2024-12-20 00:00:00,2024-12-23 00:00:00,2024-12-24 00:00:00,2024-12-25 00:00:00,2024-12-26 00:00:00,2024-12-27 00:00:00,2024-12-30 00:00:00
0,2021-01-29,S50F21C1000,4.9,11.6,7.6,10.5,17.4,13.9,12.9,13.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-29,S50F21C1025,2.5,6.6,4.0,5.7,10.1,7.4,6.6,6.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-29,S50F21C1050,1.1,3.5,2.0,2.9,5.5,3.6,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-29,S50F21C1075,NaN,NaN,NaN,NaN,NaN,1.6,1.3,1.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-29,S50F21C650,272.6,302.4,289.0,304.0,326.2,323.8,324.5,329.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409,2025-06-30,S50M25P875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5,19.5
1410,2025-06-30,S50M25P900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.7,30.0
1411,2025-06-30,S50M25P925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.5,45.1
1412,2025-06-30,S50M25P950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,62.1
